In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-detection-dataset/spam_sms.csv


# A Word-Level Neural Language Model And Generate Text

In [3]:
import pandas as pd

def load_doc_from_csv(file_path, column_name):
    """
    Load document content from a CSV file.

    Args:
    - file_path (str): The path to the CSV file.
    - column_name (str): The name of the column containing the document content.

    Returns:
    - str: Document content.
    """
    try:
        # Load CSV file
        df = pd.read_csv(file_path)
        # Extract document content from specified column
        doc = df['v2'].iloc[0]  # Assuming there's only one document in the CSV file
        return doc
    except FileNotFoundError:
        print("File not found:", file_path)
        return None

# Specify the file path and column name
file_path = '/kaggle/input/sms-spam-detection-dataset/spam_sms.csv'  # Replace 'your_csv_file.csv' with the actual file path
column_name = 'v2'  

# Load document
doc = load_doc_from_csv(file_path, column_name)
if doc is not None:
    print(doc[:200])  # Print the first 200 characters of the document

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


In [4]:
import string

# turn a doc into clean tokens
def clean_doc(doc):
    """
    Clean and tokenize a document.

    Args:
    - doc (str): The document to be cleaned.

    Returns:
    - list: List of clean tokens.
    """
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation and make lower case
    table = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(table).lower() for word in tokens if word.isalpha()]
    return tokens

In [5]:
# Clean document
tokens = clean_doc(doc)

# Print the first 200 tokens
print("First 200 tokens:")
print(tokens[:200])

# Print total number of tokens and unique tokens
total_tokens = len(tokens)
unique_tokens = len(set(tokens))
print('Total Tokens:', total_tokens)
print('Unique Tokens:', unique_tokens)

First 200 tokens:
['go', 'until', 'jurong', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'cine', 'there', 'got', 'amore']
Total Tokens: 16
Unique Tokens: 16


In [6]:
# Define the length of each sequence
sequence_length = 50 + 1

# Initialize a list to store sequences
sequences = []

# Generate sequences of tokens
for i in range(sequence_length, len(tokens)):
    # Select sequence of tokens
    sequence = tokens[i - sequence_length: i]
    # Convert the sequence into a line
    line = ' '.join(sequence)
    # Store the line
    sequences.append(line)

# Print the total number of sequences
print('Total Sequences:', len(sequences))

Total Sequences: 0


In [7]:
!pip install --upgrade jax jaxlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 MB 18.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.25
    Uninstalling jaxlib-0.4.25:
      Successfully uninstalled jaxlib-0.4.25
  Attempting uninstall: jax
    Found existing installation: jax 0.4.25
    Uninstalling jax-0.4.25:
      Successfully uninstalled jax-0.4.25


In [8]:
import pandas as pd
from numpy import array
from pickle import dump
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

2024-04-07 07:48:26.382183: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 07:48:26.382328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 07:48:26.531495: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
# Load the dataset from CSV file
file_path = '/kaggle/input/sms-spam-detection-dataset/spam_sms.csv'
df = pd.read_csv(file_path)

In [10]:
# Filter spam and ham messages
spam_df = df[df['v2'] == 'spam']
ham_df = df[df['v2'] == 'ham']

In [11]:
# Combine spam and ham messages
df = pd.concat([spam_df, ham_df])

In [13]:
# Extract text and text type columns
lines = df['v2'].tolist()

In [14]:
# Integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [15]:
# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1

> Use Language Model

In [18]:
import pandas as pd
from random import randint
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Load cleaned v2 sequences from CSV file
df = pd.read_csv('/kaggle/input/sms-spam-detection-dataset/spam_sms.csv')
lines = df['v2'].tolist()

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
vocab_size = len(tokenizer.word_index) + 1

# Generate sequences
sequences = tokenizer.texts_to_sequences(lines)
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')

# Prepare input and output sequences
X = sequences[:, :-1]
y = sequences[:, -1]

# Define the model
model = Sequential()
model.add(Embedding(vocab_size, 50))
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=50, verbose=2)

# Function to generate sequence
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = []
    in_text = seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre')
        # predict probabilities for each word
        yhat = model.predict(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat.argmax():
                out_word = word
                break
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

# Select a seed text
seed_text = lines[randint(0, len(lines))]
print(seed_text + '\n')

# Generate new text
generated = generate_seq(model, tokenizer, max_length-1, seed_text, 50)
print(generated)

Epoch 1/50
175/175 - 26s - 149ms/step - accuracy: 0.0162 - loss: 7.9998
Epoch 2/50
175/175 - 41s - 233ms/step - accuracy: 0.0153 - loss: 7.0193
Epoch 3/50
175/175 - 41s - 235ms/step - accuracy: 0.0178 - loss: 6.8540
Epoch 4/50
175/175 - 23s - 133ms/step - accuracy: 0.0185 - loss: 6.7455
Epoch 5/50
175/175 - 41s - 235ms/step - accuracy: 0.0210 - loss: 6.6424
Epoch 6/50
175/175 - 41s - 233ms/step - accuracy: 0.0210 - loss: 6.5044
Epoch 7/50
175/175 - 41s - 234ms/step - accuracy: 0.0258 - loss: 6.3441
Epoch 8/50
175/175 - 42s - 237ms/step - accuracy: 0.0370 - loss: 6.1783
Epoch 9/50
175/175 - 41s - 232ms/step - accuracy: 0.0459 - loss: 6.0202
Epoch 10/50
175/175 - 41s - 234ms/step - accuracy: 0.0537 - loss: 5.8609
Epoch 11/50
175/175 - 24s - 135ms/step - accuracy: 0.0642 - loss: 5.6993
Epoch 12/50
175/175 - 41s - 234ms/step - accuracy: 0.0754 - loss: 5.5298
Epoch 13/50
175/175 - 41s - 233ms/step - accuracy: 0.0827 - loss: 5.3839
Epoch 14/50
175/175 - 24s - 135ms/step - accuracy: 0.0933 - 